In [2]:
from dotenv import load_dotenv
load_dotenv()
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.agent.openai import OpenAIAgent
from fredapi import Fred
import os
import matplotlib.pyplot as plt
import pandas as pd
import json

from src.base import RealtorAgentToolSpec

In [3]:
agent = OpenAIAgent.from_tools(RealtorAgentToolSpec(realtor_api_key=os.environ['REALTOR_API_KEY']).to_tool_list(),
                               verbose=True)

In [16]:
response = agent.chat("summarize 10 property listings for sale near the address 2111 N 26 AVE Hollywood, Florida 33020. Include their addresses")
print(response)

Added user message to memory: summarize 10 property listings for sale near the address 2111 N 26 AVE Hollywood, Florida 33020. Include their addresses
=== Calling Function ===
Calling function: get_property_id with args: {"address": "2111 N 26 AVE Hollywood, Florida 33020"}
Got output: 6321659402

=== Calling Function ===
Calling function: get_listings_by_postal_code with args: {"postal_code": "33020"}
Got output:      __typename property_id  listing_id plan_id    status  photo_count  \
0    SearchHome  5397105889  2965448693    None  for_sale           50   
1    SearchHome  6309771566  2965445554    None  for_sale            4   
2    SearchHome  6612881370  2965437071    None  for_sale            1   
3    SearchHome  6585714497  2965425129    None  for_sale           57   
4    SearchHome  9509142861  2965420937    None  for_sale           12   
..          ...         ...         ...     ...       ...          ...   
195  SearchHome  6792675021  2963232079    None  for_sale       

In [9]:
import requests
def get_property_id(api_key, address:str) -> str:
    """Given a property address, returns a property id"""
    
    REALTOR_API_KEY = api_key["REALTOR_API_KEY"]
    url = "https://realtor.p.rapidapi.com/locations/v2/auto-complete"
    querystring = {"input":address, "limit":"10"}
    headers = {
    	"X-RapidAPI-Key": REALTOR_API_KEY,
    	"X-RapidAPI-Host": "realtor.p.rapidapi.com"
    }
    
    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code != 200:
        return ''

    location_metadata_df = pd.json_normalize(response.json()['autocomplete'])
    property_id = str(location_metadata_df[location_metadata_df['area_type'].eq('address')]['mpr_id'].values[0])
    
    return property_id


get_property_id(api_key={'REALTOR_API_KEY':os.environ['REALTOR_API_KEY']},
                address='2111 N 26 AVE Holywood, Florida 33020')

KeyError: 'mpr_id'

In [11]:
import requests

url = "https://realtor.p.rapidapi.com/locations/v2/auto-complete"

querystring = {"input":"2111 N 26 AVE hollywood florida 33020","limit":"10"}

headers = {
	"X-RapidAPI-Key": "7fb8c84c68msh069075b8bf72228p1011f5jsnd15da5f215b7",
	"X-RapidAPI-Host": "realtor.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

response.json()

{'meta': {'version': '0.109.0.0.109.0.2163.v109',
  'es_took': 13,
  'es_total_hits': 13},
 'autocomplete': [{'area_type': 'postal_code',
   '_id': 'ps:fl_33020',
   '_score': 24.821571,
   'postal_code': '33020',
   'state_code': 'FL',
   'city': 'Hollywood',
   'country': 'USA',
   'centroid': {'lon': -80.152424, 'lat': 26.019015},
   'slug_id': '33020',
   'geo_id': 'ebd1cf8c-552f-5751-9d9a-c97499d56275'},
  {'area_type': 'address',
   '_id': 'addr:6321659402',
   '_score': 21.276718,
   'mpr_id': '6321659402',
   'full_address': ['2111 N 26th Ave Hollywood FL 33020',
    '2111 N 26th Ave Hollywood FL'],
   'line': '2111 N 26th Ave',
   'city': 'Hollywood',
   'postal_code': '33020',
   'state_code': 'FL',
   'country': 'USA',
   'centroid': {'lon': -80.159277, 'lat': 26.030298},
   'prop_status': ['recently_sold', 'off_market'],
   'validation_code': ['121']},
  {'area_type': 'address',
   '_id': 'addr:9844944301',
   '_score': 21.118555,
   'mpr_id': '9844944301',
   'full_address

In [13]:
location_metadata_df = pd.json_normalize(response.json()['autocomplete'])
property_id = str(location_metadata_df[location_metadata_df['area_type'].eq('address')]['mpr_id'].values[0])
property_id

'6321659402'